### 1. Prepare the working environment

In [ ]:
HOME = "../.." # path to the root of the project two levels up

# Activate the environment
using Pkg
Pkg.activate(HOME)
Pkg.precompile()

In [ ]:
using IceFloeTracker: pairfloes, deserialize, DataFrames, colorview, RGB, PaddedView, float64, mosaicview, Gray, tagfloes!
imshow(x) = Gray.(x);


### 2. Set thresholds for the algorithm. Load data

In [ ]:
# Set thresholds
t1 = (dt=(30.0, 100.0, 1300.0), dist=(200, 250, 300))
t2 = (
    area=1200,
    arearatio=0.28,
    majaxisratio=0.10,
    minaxisratio=0.12,
    convexarearatio=0.14,
)
t3 = (
    area=10_000,
    arearatio=0.18,
    majaxisratio=0.1,
    minaxisratio=0.15,
    convexarearatio=0.2,
)
condition_thresholds = (t1, t2, t3)
mc_thresholds = (
    goodness=(area3=0.18, area2=0.236, corr=0.68), comp=(mxrot=10, sz=16)
)
dt = [15.0, 20.0]

# Load data
floedata = deserialize(joinpath(HOME, "test", "test_inputs", "tracker", "tracker_test_data.dat"))
fieldnames(typeof(floedata))

props, imgs = floedata.props, floedata.imgs;

### 3. Pre-labeling workflow

In [ ]:
# Sort by area in descending order and make an ID column for the day
for (i, prop) in enumerate(props)
    # sort by area in descending order
    sort!(prop, :area; rev=true)

    # make an ID column in props[1] with values 1:size(props)[1]
    prop[!, "ID"] = 1:size(prop)[1]

    # make a constant column "Day" in props[1] with value 1
    prop[!, "Day"] = fill(i, size(prop)[1])
end

# Preliminary pairs
_pairs = pairfloes(imgs, props, dt, condition_thresholds, mc_thresholds);


# Concatenate preliminary pairs horizontally into an array of DataFrames `pairs` with the pairings only
cols = ["Day", "ID"]
pairs = [hcat(pair.props1[:, cols], pair.props2[:, cols], makeunique=true) for (i, pair) in enumerate(_pairs)]


### 4. Tag paired floes

In [ ]:
tagfloes!(props, pairs)


### 5. Inspect paired floes

In [ ]:
# View matching pair of floes `i` in days `d-1` and `d`:
i=1; d=2; # d can be 2 (for days 1-2) or 3 (days 2-3)

In [ ]:
# Run cell several times to view different pairs
floe1 = props[d][i,["Floe_ID","mask"]]
i += 1 # increase to get next pair in the next run of the cell
floe0 = props[d-1][floe1.Floe_ID, "mask"]
floes = [floe0, floe1.mask]
# Get common canvas size for floes in case masks are different sizes and display the pair
s = Tuple(maximum(r) for r in eachrow(hcat([collect(size(floe)) for floe in floes]...)))
imshow(mosaicview([collect(PaddedView(0.0, float64.(img), s)) for img in floes], nrow=1))